In [1]:
import numpy as np
import matplotlib.pyplot as plt

import time
from copy import deepcopy

import cv2

In [2]:
%load_ext line_profiler

# Useful resources
[OpenCV tutorial](http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html#playing-video-from-file)

# Capture and handle video

In [19]:
FILE = '../test_data/smooth_1.mp4'
cv2.namedWindow('raw_frame')

In [23]:
cap = cv2.VideoCapture(FILE)

In [24]:
def handleFrame(frame, features, verbose=False):
    image = Image(frame)

    for f in features:
        f.refine(image, verbose=verbose)
    
    return image


def main():
    features = deepcopy([greenSmall, orangeSmall])
    i=0
    while cap.isOpened():
        verbose = i%50 == 0
        i += 1
        
        startTime = time.time()
        ret, frame = cap.read()
        readTime = time.time()

        if frame is None or len(frame) <= 0:
            print "End of video file reached"
            break

        # Do work here
        image = handleFrame(frame, features, verbose=verbose)
        
        finishTime = time.time()

        for f in features:
            f.draw(frame)
            
        image.draw(frame)
        scaled = cv2.resize(frame, dsize=None, fx=0.5, fy=0.5)
        cv2.imshow('raw_frame', scaled)

        cv2.waitKey(10)
        
        if verbose:
            print "{0:.1f}ms reading image, {1:.1f}ms processing, {2}px accessed\n".format(
                (readTime-startTime)*1000, 
                (finishTime-readTime)*1000, 
                len(image.pixelsAccessed))
        
main()

(1100, 700)
Left edge found after 192 pixels
Right edge found after 39 pixels
Top edge found after 93 pixels
Bottom edge found after 145 pixels
(600, 700)
Center point color is [ 22  24 205]
26.6ms reading image, 7.8ms processing, 476px accessed

(1158, 792)
Left edge found after 124 pixels
Right edge found after 131 pixels
Top edge found after 123 pixels
Bottom edge found after 130 pixels
(570, 759)
Left edge found after 112 pixels
Right edge found after 119 pixels
Top edge found after 117 pixels
Bottom edge found after 124 pixels
4.3ms reading image, 13.9ms processing, 990px accessed

(1064, 831)
Left edge found after 132 pixels
Right edge found after 132 pixels
Top edge found after 132 pixels
Bottom edge found after 131 pixels
(419, 800)
Left edge found after 133 pixels
Right edge found after 130 pixels
Top edge found after 132 pixels
Bottom edge found after 131 pixels
4.1ms reading image, 14.7ms processing, 1063px accessed

(1172, 793)
Left edge found after 134 pixels
Right edge fo

# Profile things

In [ ]:
ret, frame = cap.read()

def bench():
    features = deepcopy([greenCircle])
    handleFrame(frame, features)
    
%lprun -r -f Circle2D.refine bench()

# Define types for features

In [4]:
class Image(object):
    def __init__(self, img):
        self.img = img
        self.pixelsAccessed = []
    
    def getHSV(self, x, y):
        self.pixelsAccessed.append((x, y))
        rgb = self.img[y:y+1, x:x+1]
        hsv = cv2.cvtColor(rgb, cv2.COLOR_BGR2HSV)
        return hsv[0, 0]
    
    def draw(self, img):
        color = (0, 0, 255)
        for p in self.pixelsAccessed:
            img[p[1], p[0]] = color

In [5]:
class Color(object):
    def __init__(self, minimum, maximum, colorspace='HSV'):
        if colorspace != 'HSV':
            raise Exception("Invalid colorspace, only HSV supported")
            
        self.min = minimum
        self.max = maximum
        
    def matches(self, color):
        # Check hue condition
        if self.min[0] > self.max[0]:
            # The hue range wraps past the edge
            if color[0] > self.max[0] and color[0] < self.min[0]:
                return False
        else:
            if color[0] > self.max[0] or color[0] < self.min[0]:
                return False
            
        # Check saturation and value conditions
        for i in (1, 2):
            if color[i] > self.max[i] or color[i] < self.min[i]:
                return False
        
        return True

In [6]:
class Circle2D(object):
    def __init__(self, center, radius, colorRange):
        self.center = center
        self.radius = radius
        self.color  = colorRange
        
        # TODO: this should be a scalar with more reasonable behavior
        self.confidence = True
    
    def draw(self, img):
        """
        Taking in a numpy array representing an OpenCV image,
        draws a visual debugging indication of this feature
        """
        drawColor = (128, 255, 0) if self.confidence else (0, 0, 255)
        cv2.circle(img, self.center, self.radius, drawColor, thickness=5)
        cv2.circle(img, self.center, 5, drawColor, thickness=-1)
        
    def refine(self, image, verbose=False, searchRange = 200):
        """
        Taking in an Image object, updates this feature to better align
        with the observed pixels.
        
        searchRange represents the maximum pixel distance the target
            object could have moved.
        """
        
        def matchesAt(x, y):
            return self.color.matches(image.getHSV(x, y))
        
        
        if verbose:
            print self.center

            
        
        if matchesAt(*self.center):
            self.confidence = True
            
        else:
            self.confidence = False
            if verbose:
                print "Center point color is {}".format(
                    image.getHSV(*self.center))
            return
        
        NOISE_DIST = 15
        
        ## Find the x-coordinate of the center of the circle
        leftEdge = rightEdge = self.center[0]
        
        maxDist = searchRange + self.radius
        
        missCount = 0
        for i in range(maxDist):
            if not matchesAt(self.center[0]-i, self.center[1]):
                missCount += 1
            else:
                missCount = 0
            
            if missCount > NOISE_DIST:
                leftEdge = self.center[0]-(i - NOISE_DIST)
                if verbose:
                    print "Left edge found after {} pixels".format(i)
                break
        
        missCount = 0
        for i in range(maxDist):
            if not matchesAt(self.center[0]+i, self.center[1]):
                missCount += 1
            else:
                missCount = 0
            
            if missCount > NOISE_DIST:
                rightEdge = self.center[0]+(i - NOISE_DIST)
                if verbose:
                    print "Right edge found after {} pixels".format(i)
                break
                
        self.center = (int((leftEdge + rightEdge) / 2), self.center[1])
        
        # Find the vertical position
        
        topEdge = bottomEdge = self.center[1]
        
        maxDist = searchRange + self.radius
        
        missCount = 0
        for i in range(maxDist):
            if not matchesAt(self.center[0], self.center[1]-i):
                missCount += 1
            else:
                missCount = 0
            
            if missCount > NOISE_DIST:
                topEdge = self.center[1]-(i - NOISE_DIST)
                if verbose:
                    print "Top edge found after {} pixels".format(i)
                break
        
        missCount = 0
        for i in range(maxDist):
            if not matchesAt(self.center[0], self.center[1]+i):
                missCount += 1
            else:
                missCount = 0
            
            if missCount > NOISE_DIST:
                bottomEdge = self.center[1]+(i - NOISE_DIST)
                if verbose:
                    print "Bottom edge found after {} pixels".format(i)
                break
                
        self.center = (self.center[0], int((topEdge + bottomEdge) / 2))
        
        self.radius = abs(int((topEdge - bottomEdge) / 2))
        
        
        
        


In [7]:
green = Color((30, 60, 100), (60, 255, 255))
orange = Color((5, 150, 130), (30, 255, 255))

greenCircle = Circle2D((1920/2, 1080/2), 400, green)
orangeCircle = Circle2D((1920/2, 1080/2), 400, orange)

greenSmall = Circle2D((1100, 700), 130, green)
orangeSmall = Circle2D((600, 700), 130, orange)